In [10]:
from constraint import *
import time
from random import randrange
import numpy as np

In [11]:
problem = Problem()

In [12]:
for i in range(12):
    problem.addVariable(str(i), [200,400])

In [13]:
problem.addConstraint(ExactSumConstraint(3600), ("0", "1" ,"2", "3", "4", "5", "6", "7", "8", "9", "10", "11"))
problem.addConstraint(ExactSumConstraint(1800), ("0", "1" ,"2", "3", "4", "5"))
problem.addConstraint(ExactSumConstraint(1800), ("6", "7", "8", "9", "10", "11"))

In [14]:
james = problem.getSolutions()

In [15]:
len(james)

400

In [84]:
np.save('csp', james) 

In [5]:
james = np.load('csp.npy',allow_pickle=True) 

In [6]:
import numpy as np
import mido
import tensorflow as tf

from keras.models import load_model
from mido import MidiFile, MidiTrack, Message


model = load_model('classical.h5')

Using TensorFlow backend.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [7]:
#Midi Notes range from 0 - 127
#Input 4 notes for a chord progression
#user_input = [[[27], [66], [81], [2]]]
user_input = [[[23], [99], [2], [5]]]
user_input = np.array(user_input)
#reshaping input to 3D
user_input.reshape(1,1,4)

array([[[23, 99,  2,  5]]])

In [8]:
#Performing inference on the model
predict = model.predict_on_batch(user_input)
#Adjusting from normalization
predict = predict * 11
predict = abs(predict)
predict.astype(int)

array([[[19],
        [21],
        [22],
        [23],
        [18],
        [16],
        [16],
        [16],
        [16],
        [15],
        [14],
        [14]]])

In [9]:
#helper function to classify notes into bins
def note_reg(note):
    C = [0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120]
    C_sharp = [1, 13, 25, 37, 49, 61, 73, 85, 97, 109, 121]
    D = [2, 14, 26, 38, 50, 62, 74, 86, 98, 110, 122]
    D_sharp = [3, 15, 27, 39, 51, 63, 75, 87, 99, 111, 123]
    E = [4, 16, 28, 40, 52, 64, 76, 88, 100, 112, 124]
    F = [5, 17, 29, 41, 53, 65, 77, 89, 101, 113, 125]
    F_sharp = [6, 18, 30, 42, 54, 66, 78, 90, 102, 114, 126]
    G = [7, 19, 31, 43, 55, 67, 79, 91, 103, 115, 127]
    G_sharp = [8, 20, 32, 44, 56, 68, 80, 92, 104, 116]
    A = [9, 21, 33, 45, 57, 69, 81, 93, 105, 117]
    A_sharp = [10, 22, 34, 46, 58, 70, 82, 94, 106, 118]
    B = [11, 23, 35, 47, 59, 71, 83, 95, 107, 119]
    if (note in C):
        return 0
    elif(note in C_sharp):
        return 1
    elif(note in D):
        return 2
    elif(note in D_sharp):
        return 3
    elif(note in E):
        return 4
    elif(note in F):
        return 5
    elif(note in F_sharp):
        return 6
    elif(note in G):
        return 7
    elif(note in G_sharp):
        return 8
    elif(note in A):
        return 9
    elif(note in A_sharp):
        return 10
    elif(note in B):
        return 11
    else:
        return 0

In [10]:
#new midi file
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

#scalable octave variable
octave = 12 * 6
t = randrange(len(james))

track.append(Message('program_change', program=1, time=0))
for x in range(5):
    for i in range(12):
        track.append(Message('note_on', note= octave + note_reg(int(predict[0][i][0])), velocity=64, time=25))
        track.append(Message('note_off', note= octave + note_reg(int(predict[0][i][0])), velocity=64, time=james[t].get(str(i))))
mid.save('song.mid')

In [ ]:
time=james[t].get(str(i)))